# Introduction

* **Description**: As an input, this script takes a zip file containing 744 csv files with ancient Greek inscriptions from the PHI database. It merges these files into one pandas dataframe and export this dataframe as a json file.
* data sources:
  * sciencedata.dk - `SDAM_root/SDAM_data/PHI`
  * gsheet "PHI_overview"
* inputs: 
  * `PHI-raw-csv-2020-08-24.zip`: 742 csv files
* outputs:
  * `PHI_merged_[timestamp].json`: full dataset
  * `PHI_sample_[timestamp].json`: first 10000 inscriptions
  * `inscriptions_list_sorted.json`: list of all inscription IDs

* **Author**: Vojtěch Kaše

* **Last complete run**: 2020-08-24 (Petra Hermankova)

# Requirements

In [ ]:
import numpy as np
import math
import pandas as pd

import sys
### we do a lot of requests during the scrapping. Some of them with requests package, some of them with urllib
import requests
from urllib.request import urlopen 
from urllib.parse import quote  
from bs4 import BeautifulSoup
import xml.etree.cElementTree as ET

import zipfile
import io

# to avoid errors, we sometime use time.sleep(N) before retrying a request
import time
# the input data have typically a json structure
import json
import getpass

import datetime as dt

from google.colab import auth
from oauth2client.client import GoogleCredentials

import gspread
from gspread_dataframe import get_as_dataframe, set_with_dataframe

In [ ]:
!pip install sddk
import sddk

# authentication

In [ ]:
conf = sddk.configure("SDAM_root", "648597@au.dk")

sciencedata.dk username (format '123456@au.dk'): 648560@au.dk
sciencedata.dk password: ··········
connection with shared folder established with you as its ordinary user
endpoint variable has been configured to: https://sciencedata.dk/sharingout/648597%40au.dk/SDAM_root/


In [ ]:
### authorize google sheets
auth.authenticate_user()
gc = gspread.authorize(GoogleCredentials.get_application_default())
# establish connection with particular sheet by its url:
PHI_overview = gc.open_by_url("https://docs.google.com/spreadsheets/d/1zfTw0Hf304maBmrYvaMxRLnv1zfAVFixrtGTTsLCcT4/edit?usp=sharing")

# Extract the data from a zipfile on sciencedata.dk


In [ ]:
# to get the zip file into a response
response = conf[0].get(conf[1] + "SDAM_data/PHI/PHI-raw-csv-2020-08-24.zip")

In [ ]:
# check that the response is functional
response

<Response [200]>

In [ ]:
# approach the response as a zip object
zipped = zipfile.ZipFile(io.BytesIO(response.content))

In [ ]:
# how many files we have?
namelist = zipped.namelist()[1:] ### index "0" is for main directory
len(namelist)

744

In [ ]:
namelist[:10]

['ZPE.csv',
 'ZMNP.csv',
 'Ziebarth--Neue-Verfluchungstafeln.csv',
 'ZbornAMS.csv',
 'YClS.csv',
 'WZHalle.csv',
 'WS.csv',
 'Wilhelm--Neue-Beitr-ge.csv',
 'Wilhelm--Beitr-ge.csv',
 'Waelkens--T-rsteine.csv']

In [ ]:
# filter for non-hidden & .csv files
namelist = [name for name in namelist if not "._" in name]
namelist = [name for name in namelist if  ".csv" in name]
len(namelist)

744

# Test with one file

In [ ]:
test = pd.read_csv(io.BytesIO(zipped.read("CEG.csv")), sep="@")

In [ ]:
test.head()

,URL,Book,Text,hdr1,hdr2,tildeinfo,note,lines,metadata,data
0,/text/328151?location=1703&patt=&bookid=303&of...,CEG,I 109,Regions,CEG\nI 109,Boiotia — Tanagra — ca. 600 BC — IG VII 579 — ...,Funerary epigram for Dermys and Kitylos. Lime...,5,"i, left figure.1\n\nii, right figure.2\n\niii,...",Δέρμυς.\n\nΚιτύλος.\n\nἈμφάλκες ἔστασ’ ἐπὶ Κιτ...
1,/text/328152?location=1703&patt=&bookid=303&of...,CEG,I 110,Regions,CEG\nI 110,Boiotia — Haliartos (Kastri Maziou) — ca. 500 ...,"Funerary epigram for Kallias, son of Aigisthos...",4,1\n\n\n,Καλλία\nΑἰγίθοιο {²⁶Αἰγίσθοιο}²⁶·\nτὺ δ’ εὖ πρ...
2,/text/328153?location=1703&patt=&bookid=303&of...,CEG,I 111,Regions,CEG\nI 111,Boiotia — Tanagra — ca. 500 BC? — IG VII 3501 ...,Funerary epigram for Gathon. Limestone tablet...,1,1,[π]αῖδες̣ ἐ̣[ποίεσ]α̣ν μ̣νᾶ̣μ̣’ ἐνθάδ̣[ε] πατρ...
3,/text/328154?location=1703&patt=&bookid=303&of...,CEG,I 112,Regions,CEG\nI 112,Boiotia — Thisbe — ca. 500 BC? — IG VII 2247 —...,Funerary epigram for Phanes. Funerary stele. ...,2,1\n,ἀσστοῖ[ς] καὶ χσένοισι Φάνες φίλος [ἐνθάδε κεῖ...
4,/text/328155?location=1703&patt=&bookid=303&of...,CEG,I 113,Regions,CEG\nI 113,Boiotia — Thespiai — Palaiopanagia — ca. 500-4...,"Funerary epigram for Oligeides, son of Osthilo...",6,1\n\n\n\n5\n,μνᾶμ’ ἐπ’ Ὀλιγέ-\nδαι {ι} μ’ ὁ πατὲρ ἐ-\nπέθεκ...


# Extracting the whole dataset

In [ ]:
dfs_list = []
for filename in namelist:
  try:
    df = pd.read_csv(io.BytesIO(zipped.read(filename)))
    if len(df.columns)<9:
      df = pd.read_csv(io.BytesIO(zipped.read(filename)), sep="@")
    if len(df.columns)<9:
      print(filename)
  except:
    df = pd.read_csv(io.BytesIO(zipped.read(filename)), sep="@")
  df["filename"] = filename
  df["PHI_ID"] = df.apply(lambda row: int(row["URL"].partition("text/")[2].partition("?")[0]), axis=1)
  dfs_list.append(df)

In [ ]:
PHI_raw = pd.concat(dfs_list)
PHI_raw.shape
# with the previous version, we had (214995, 12) / then we had 216702,14 (20 June 2020)

(218162, 12)

In [ ]:
# set index based on PHI_ID (for clarity, maintain the PHI_ID in independent column)
PHI_raw.index = PHI_raw["PHI_ID"].tolist()
PHI_raw.sort_index(inplace=True)
PHI_raw.head(5)

,URL,Book,Text,hdr1,hdr2,tildeinfo,note,lines,metadata,data,filename,PHI_ID
1,/text/1?location=1701&patt=&bookid=4&offset=0&...,IG I³,1,Regions\nAttica (IG I-III),IG I³\n1,Att. — Ath.: Akr. — stoich. 35 — c. 510-500 a....,NaN,12,1\n\n\n\n5\n\n\n\n\n10\n\n,ἔδοχσεν το͂ι δέμοι· τ̣[ὸς ἐ Σ]αλαμ̣[ῖνι κλερόχ...,IG-I³.csv,1
2,/text/2?location=1701&patt=&bookid=4&offset=0&...,IG I³,2,Regions\nAttica (IG I-III),IG I³\n2,Att. — non-stoich. — c. 500 a.,NaN,14,1\n\n\n\n5\n\n\n\n\n10\n\n\n\n,[․․8-9․․․]ν̣ βολ — — — — — — — — — —\n[․6-7․․]...,IG-I³.csv,2
3,/text/3?location=1701&patt=&bookid=4&offset=0&...,IG I³,3,Regions\nAttica (IG I-III),IG I³\n3,Att. — stoich. 21 — 490-480 a.,NaN,13,1\n\n\n\n5\n\n\n\n\n10\n\n\n,[․]αρ[․․․․]ι ℎερακλειο[․․5․․]\n[․]αρ̣ο#⁷[․] τι...,IG-I³.csv,3
4,/text/4?location=1701&patt=&bookid=4&offset=0&...,IG I³,4,Regions\nAttica (IG I-III),IG I³\n4,Att. — stoich. 38 — 485/4 a.,NaN,56,face A.1\n\n\n\n5\n\n\n\n\n10\n\n\n\n\n15\n\n\...,[․․․․․․․․․․․․․․․․․․38․․․․․․․․․․․․․․․․․․]\n[․․․...,IG-I³.csv,4
5,/text/5?location=1701&patt=&bookid=4&offset=0&...,IG I³,5,Regions\nAttica (IG I-III),IG I³\n5,Att. — c. 500 a.,NaN,6,1\n\n\n\n5\n,[ἔδοχσε]ν [⋮ τε͂ι βολε͂ι] ⋮ καὶ [τ]ο͂ι δέμοι ⋮...,IG-I³.csv,5


In [ ]:
sddk.write_file("SDAM_data/PHI/PHI_merged_2020-08-24.json", PHI_raw, conf)

A file with the same name ("PHI_merged_2020-08-24.json") already exists in this location.
Press Enter to overwrite it or choose different path and filename: 
Your <class 'pandas.core.frame.DataFrame'> object has been succefully written as "https://sciencedata.dk/sharingout/648597%40au.dk/SDAM_root/SDAM_data/PHI/PHI_merged_2020-08-24.json"


In [ ]:
# to test reading:
#PHI = sddk.read_file("SDAM_data/PHI/PHI_merged_2020-08-24.json", "df", conf)
#PHI.head(5)

# Exporting list of inscriptions ans sample


In [ ]:
PHI = sddk.read_file("SDAM_data/PHI/PHI_merged_2020-08-24.json", "df", conf)

In [ ]:
inscriptions_list = PHI.index.tolist()
print(inscriptions_list)

['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99', '100', '101', '102', '103', '104', '105', '106', '107', '108', '109', '110', '111', '112', '113', '114', '115', '116', '117', '118', '119', '120', '121', '122', '123', '124', '125', '126', '127', '128', '129', '130', '131', '132', '133', '134', '135', '136', '137', '138', '139', '140', '141', '142', '143', '144', '145', '146', '147', '148', '149', '150', '151', '152', '153', '154', '155', '156', '157', '158', '

In [ ]:
sddk.write_file("SDAM_data/PHI/inscriptions_list_sorted_2020-08-24.json", inscriptions_list, conf)

Your <class 'list'> object has been succefully written as "https://sciencedata.dk/sharingout/648597%40au.dk/SDAM_root/SDAM_data/PHI/inscriptions_list_sorted_2020-08-24.json"


In [ ]:
### there was an update in the sddk package, which now manages encoding on its own
sddk.write_file("SDAM_data/PHI/PHI_sample_2020-08-24.json", PHI[:10000], conf)

Your <class 'pandas.core.frame.DataFrame'> object has been succefully written as "https://sciencedata.dk/sharingout/648597%40au.dk/SDAM_root/SDAM_data/PHI/PHI_sample_2020-08-24.json"


# Testing: reading sample digested by R by to Python

In [ ]:
# reading the json processed in R back to Python:
PHI_sample = sddk.read_file("SDAM_data/PHI/PHI_tibble.json", "df", conf)
PHI_sample.head(5)

,URL,Book,Text,hdr1,hdr2,tildeinfo,note,lines,metadata,data,region,stoich,sinistr,subregion,date,other_info_0,other_info_1,other_info_2,other_info_3,other_info_4
0,[/text/237766?location=7&patt=&bookid=409&book...,"[Clinton, Sacred Officials]",[10],[Regions\nAttica (IG I-III)],"[Clinton, Sacred Officials 10]",{},{},[46],[5\n\n\n\n\n10\n\n\n\n\n15\n\n\n\n\n20\n\n\n\n...,[[․․․․․12․․․․․] ὀβολ[ὸν ․․5․․]\n[․․․․․12․․․․․]...,[Att.],[stoich. 23],{},[Ath.: Eleusinion],[ca. 460 BC],[ca. 460 BC],"[IG I² 6,C]","[I(3).6,C ll. 5-50]",{},{}
1,[/text/237767?location=7&patt=&bookid=409&book...,"[Clinton, Sacred Officials]","[18,H5]",[Regions\nAttica (IG I-III)],"[Clinton, Sacred Officials 18,H5]",{},{},[35],[\n1\n\n\n\n5\n\n\n\n\n10\n\n\n\n\n15\n\n\n\n\...,[corona\nθ[εοί].\nΕ̣ὐθ̣[․․6․․․]θ̣ωνος Ἐλευσίνι...,[Att.],[stoich. 25],{},[Eleusis],[ca. mid-4th c. BC],[ca. mid-4th c. BC],[IG II² 1188],{},{},{}
2,[/text/237768?location=7&patt=&bookid=409&book...,"[Clinton, Sacred Officials]","[24,H11]",[Regions\nAttica (IG I-III)],"[Clinton, Sacred Officials 24,H11]",{},{},[26],[1\n\n\n\n5\n\n\n\n\n10\n\n\n\n\n15\n\n\n\n\n2...,[ἐπὶ Λυσιάδου ἄρχοντος Π̣[υανοψ]ιῶνος ἕκτ[ει ἐ...,[Att.],[non-stoich.],{},[Ath.: Agora],{},[148/7? BC],"[Hsp.11.1942.293,58 [SEG 19.124]]",{},{},{}
3,[/text/237769?location=7&patt=&bookid=409&book...,"[Clinton, Sacred Officials]","[30,H19]",[Regions\nAttica (IG I-III)],"[Clinton, Sacred Officials 30,H19]",{},{},[5],[1\n\n\n\n5],[[Τ. Φλ]ά̣β. Πάντα[ινον]\n[Γαργ]ήττιον vac.\n[...,[Att.],{},{},[Athens],[ca. 100 AD?],[ca. 100 AD?],"[Hsp.35.1966.247,8 +]","[AD 25.1970.187,6 [SEG 23.114]]",[SEG 29.180],{}
4,[/text/237770?location=7&patt=&bookid=409&book...,"[Clinton, Sacred Officials]","[31,H20]",[Regions\nAttica (IG I-III)],"[Clinton, Sacred Officials 31,H20]",{},{},[14],[1\n\n\n\n5\n\n\n\n\n10\n\n\n\n],[[κ]α̣θ’ ὑπομνη[ματι]-\nσμὸν τῆς ἐξ Ἀ[ρείου]\n...,[Att.],{},{},[Eleusis],[166/7 AD],[166/7 AD],[IG II² 3984],{},{},{}
